이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.5.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=281845512 sha256=ab316fa9de4d46b25066dfe942c5c8dce71a511b10beeec1507edf12ac6dab90
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


**Spark Session**을 하나 만든다

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark Writing Demo") \
        .master("local[3]") \
        .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.3.1") \
        .getOrCreate()

In [3]:
!wget https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv

--2023-01-26 23:15:42--  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv
Resolving pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)... 3.5.81.204, 3.5.84.192, 52.92.241.194, ...
Connecting to pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)|3.5.81.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/csv]
Saving to: ‘appl_stock.csv’

appl_stock.csv      100%[===================>] 139.78K   837KB/s    in 0.2s    

2023-01-26 23:15:43 (837 KB/s) - ‘appl_stock.csv’ saved [143130/143130]



In [4]:
df = spark.read \
    .format("csv") \
    .load("appl_stock.csv")

In [5]:
print("Num Partitions before: " + str(df.rdd.getNumPartitions()))
df.groupBy(spark_partition_id()).count().show()

Num Partitions before: 1
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0| 1763|
+--------------------+-----+



In [6]:
df2 = df.repartition(4)
print("Num Partitions after: " + str(df2.rdd.getNumPartitions()))
df2.groupBy(spark_partition_id()).count().show()

Num Partitions after: 4
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  440|
|                   1|  441|
|                   2|  441|
|                   3|  441|
+--------------------+-----+



In [7]:
df3 = df2.coalesce(2)
print("Num Partitions after: " + str(df3.rdd.getNumPartitions()))
df3.groupBy(spark_partition_id()).count().show()

Num Partitions after: 2
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  881|
|                   1|  882|
+--------------------+-----+



In [8]:
df.write \
    .format("avro") \
    .mode("overwrite") \
    .option("path", "dataOutput/avro/") \
    .save()

In [9]:
df2.write \
    .format("parquet") \
    .mode("overwrite") \
    .option("path", "dataOutput/parquet/") \
    .save()

In [10]:
df3.write \
    .format("json") \
    .mode("overwrite") \
    .option("path", "dataOutput/json/") \
    .save()

In [11]:
!ls -tl dataOutput/

total 12
drwxr-xr-x 2 root root 4096 Jan 26 23:31 json
drwxr-xr-x 2 root root 4096 Jan 26 23:31 parquet
drwxr-xr-x 2 root root 4096 Jan 26 23:31 avro


In [12]:
!ls -tl dataOutput/parquet/

total 112
-rw-r--r-- 1 root root     0 Jan 26 23:31 _SUCCESS
-rw-r--r-- 1 root root 28024 Jan 26 23:31 part-00003-a272730d-6f75-4d77-85b4-a0bc74a40904-c000.snappy.parquet
-rw-r--r-- 1 root root 27787 Jan 26 23:31 part-00001-a272730d-6f75-4d77-85b4-a0bc74a40904-c000.snappy.parquet
-rw-r--r-- 1 root root 27833 Jan 26 23:31 part-00000-a272730d-6f75-4d77-85b4-a0bc74a40904-c000.snappy.parquet
-rw-r--r-- 1 root root 28069 Jan 26 23:31 part-00002-a272730d-6f75-4d77-85b4-a0bc74a40904-c000.snappy.parquet


In [13]:
!ls -tl dataOutput/avro/

total 88
-rw-r--r-- 1 root root     0 Jan 26 23:31 _SUCCESS
-rw-r--r-- 1 root root 89029 Jan 26 23:31 part-00000-102346ea-bf2b-4535-98bc-72f26c556ee9-c000.avro


In [14]:
!ls -tl dataOutput/json/

total 240
-rw-r--r-- 1 root root      0 Jan 26 23:31 _SUCCESS
-rw-r--r-- 1 root root 122570 Jan 26 23:31 part-00000-7b59cfdd-9394-4373-8445-5014c08f27cb-c000.json
-rw-r--r-- 1 root root 122814 Jan 26 23:31 part-00001-7b59cfdd-9394-4373-8445-5014c08f27cb-c000.json
